In [2]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken


In [2]:
COMPLETIONS_MODEL = "text-curie-001"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [3]:
openai.api_key = 'OPENAI-API-KEY'

In [55]:
prompt = "Who is John Galt?"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"John Galt is a fictional character in Ayn Rand's novel Atlas Shrugged. He is the protagonist and the main antagonist of the novel."

<h2>Slight Example</h2>

In [13]:
prompt = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

Context:
The men's high jump event at the 2020 Summer Olympics took place between 30 July and 1 August 2021 at the Olympic Stadium.
33 athletes from 24 nations competed; the total possible number depended on how many nations would use universality places 
to enter athletes in addition to the 32 qualifying through mark or ranking (no universality places were used in 2021).
Italian athlete Gianmarco Tamberi along with Qatari athlete Mutaz Essa Barshim emerged as joint winners of the event following
a tie between both of them as they cleared 2.37m. Both Tamberi and Barshim agreed to share the gold medal in a rare instance
where the athletes of different nations had agreed to share the same medal in the history of Olympics. 
Barshim in particular was heard to ask a competition official "Can we have two golds?" in response to being offered a 
'jump off'. Maksim Nedasekau of Belarus took bronze. The medals were the first ever in the men's high jump for Italy and 
Belarus, the first gold in the men's high jump for Italy and Qatar, and the third consecutive medal in the men's high jump
for Qatar (all by Barshim). Barshim became only the second man to earn three medals in high jump, joining Patrik Sjöberg
of Sweden (1984 to 1992).

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'Gianmarco Tamberi and Mutaz Essa Barshim both won the gold medal in the event.'

<h2> Start of step 2 </h2>

In [147]:
# We have hosted the processed dataset, so you can download it directly without having to recreate it.
# This dataset has already been split into sections, one row for each section of the Wikipedia page.

df = pd.read_csv('art_test.csv')
#df = pd.read_csv('olympics_sections_text.csv')
df = df.set_index(["title", "heading"])
print(f"{len(df)} rows in the data.")
df.sample(5)

9 rows in the data.


,,content,tokens
title,heading,,
Questions and answers - Customizing URLs,Url,"Your overall profile, artwork, and portfolios ...",69
Questions and answers - Why Join Pro,Join Pro,Pro members get access to many exclusive benef...,26
Questions and answers - Pro members,Summary,A personal Showroom to sell their original and...,52
Questions and answers - Art and design lovers,Summary,Discover works in over 75 categories and style...,83
Questions and answers - Expert Selected,Summary,Each artwork is hand-picked by our seasoned cu...,20


In [149]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

In [150]:
def load_embeddings(fname: str) -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

In [180]:
#Calculate Embeddings

document_embeddings = compute_doc_embeddings(df)

In [152]:
# An example embedding:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

('Questions and answers - Why Join Pro', 'Join Pro') : [-0.01421249657869339, -0.040983621031045914, -0.012888058088719845, -0.07185579836368561, -0.02936621755361557]... (1536 entries)


<h2>2) Find the most similar document embeddings to the question embedding</h2>
<h3>At the time of question-answering, to answer the user's query we compute the query embedding of the question and use it to find the most similar document sections. Since this is a small example, we store and search the embeddings locally. Since we have a larger dataset, we should consider using a vector search engine like Pinecone to power the search.</h3>

In [153]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [154]:
order_document_sections_by_query_similarity("Pro member", document_embeddings)[:5]

[(0.8949881079474454, ('Questions and answers - Why Join Pro', 'Join Pro')),
 (0.759366675261, ('Questions and answers - Pro members', 'Summary')),
 (0.7559102038333371, ('Questions and answers - Artist centered', 'Summary')),
 (0.7498794704228923,
  ('Questions and answers - Commercial Curation Services', 'Summary')),
 (0.7426365542430916, ('Questions and answers - Quality Reviewed', 'Summary'))]

<h2>3) Add the most relevant document sections to the query prompt</h2>
<h3>Once we've calculated the most relevant pieces of context, we construct a prompt by simply prepending them to the supplied query. It is helpful to use a query separator to help the model distinguish between separate pieces of text.<h/3>

In [156]:
MAX_SECTION_LEN = 300
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [157]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [158]:
prompt = construct_prompt(
    "How do I become a member?",
    document_embeddings,
    df
)

print("===\n", prompt)

Selected 6 document sections:
('Questions and answers - Why Join Pro', 'Join Pro')
('Questions and answers - Customizing URLs', 'Url')
('Questions and answers - Art and design lovers', 'Summary')
('Questions and answers - Pro members', 'Summary')
('Questions and answers - Commercial Curation Services', 'Summary')
('Questions and answers - Artist centered', 'Summary')
===
 Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

* Pro members get access to many exclusive benefits. Special selling opportunities, priority consideration for commercial projects, paid programs, and professional development.
* Your overall profile, artwork, and portfolios can each have a custom friendly URL. When you join Artrepreneur, you’ll have the option of customizing your Profile URL. You can do so by going to your profile page and clicking the pencil icon in the upper right to edit the summary sect

<h2>4) Answer the user's question based on the context.</h2>
<h3>Now that we've retrieved the relevant context and constructed our prompt, we can finally use the Completions API to answer the user's query.</h3>


In [172]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [173]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [174]:
answer_query_with_context("Who do you help?", df, document_embeddings)

Selected 7 document sections:
('Questions and answers - Commercial Curation Services', 'Summary')
('Questions and answers - Artist centered', 'Summary')
('Questions and answers - Why Join Pro', 'Join Pro')
('Questions and answers - Art and design lovers', 'Summary')
('Questions and answers - Expert Selected', 'Summary')
('Questions and answers - Quality Reviewed', 'Summary')
('Questions and answers - Pro members', 'Summary')


"We help businesses, architects, designers, and organizations with permanent installations, exhibitions, real estate projects, and marketing programs that promote your brand or purpose globally.\n\nYour purchase directly supports working artists, thanks to Artrepreneur's 0 commission pledge.\n\nPro members get access to many exclusive benefits. Special selling opportunities, priority consideration for commercial projects, paid programs, and professional development.\n\nDiscover works in over 75 categories and styles from over 100 thousand artists, designers, and makers across the globe. Hire art & design talent for their next creative project. Subscribe to our YouTube channel, showcasing artful careers and notable visual art and design works.\n\nFollow our social media pages on Instagram, Facebook, Linked In, Twitter, and Pinterest. Become part of a global art community to show support for great artists worldwide!\n\nEach artwork is hand-picked by our seasoned curatorial team to bring 

<h3>More Examples</h3>

In [175]:
query = "Do you have curators?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 6 document sections:
('Questions and answers - Expert Selected', 'Summary')
('Questions and answers - Art and design lovers', 'Summary')
('Questions and answers - Commercial Curation Services', 'Summary')
('Questions and answers - Pro members', 'Summary')
('Questions and answers - Customizing URLs', 'Url')
('Questions and answers - Artist centered', 'Summary')

Q: Do you have curators?
A: Each artwork is hand-picked by our seasoned curatorial team to bring you the best works from around the world.

Yes, we have curators who select the works that will be featured on our website.


In [181]:
query = "How do I make a custom URL?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 6 document sections:
('Questions and answers - Customizing URLs', 'Url')
('Questions and answers - Pro members', 'Summary')
('Questions and answers - Commercial Curation Services', 'Summary')
('Questions and answers - Art and design lovers', 'Summary')
('Questions and answers - Why Join Pro', 'Join Pro')
('Questions and answers - Quality Reviewed', 'Summary')

Q: How do I make a custom URL?
A: You can make a custom URL by going to your profile page and clicking the pencil icon in the upper right to edit the summary section. Then you can add your customized URL as shown below.


In [179]:
query = "What is a personal showroom?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 7 document sections:
('Questions and answers - Pro members', 'Summary')
('Questions and answers - Commercial Curation Services', 'Summary')
('Questions and answers - Expert Selected', 'Summary')
('Questions and answers - Quality Reviewed', 'Summary')
('Questions and answers - Why Join Pro', 'Join Pro')
('Questions and answers - Art and design lovers', 'Summary')
('Questions and answers - Artist centered', 'Summary')

Q: What is a personal showroom?
A: A personal showroom is a space where an individual can sell their original and limited edition works. There is no approval required for this type of business. Priority placement in the Art Entrepreneur browse and search is available to personal showrooms. Exclusive promotional, learning, and professional development opportunities are also available on Artrepreneur and popular social media platforms. Indexing on public search engines, such as Google, Bing, and Yahoo, is available to personal showrooms.


-------------------------------------------------------------------------------------------------------------------